# Análisis Exploratorio de Datos - Opiniones Turísticas

Este notebook realiza un análisis exploratorio de los datos de opiniones turísticas recopilados de diferentes ciudades y atracciones en México.

## Objetivo
- Cargar y consolidar datos de todas las ciudades
- Realizar análisis básico de calidad de datos
- Explorar características generales del dataset

In [560]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configuración para gráficos
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

## 1. Carga y Procesamiento de Datos

Vamos a cargar todos los archivos CSV de las diferentes ciudades y crear un dataset consolidado.

In [561]:
# Función para extraer el nombre de la atracción del nombre del archivo
def extraer_nombre_atraccion(nombre_archivo):
    """
    Extrae el nombre de la atracción del nombre del archivo CSV.
    Por ejemplo: 'cancun-la-isla.csv' -> 'la isla'
    """
    # Remover la extensión .csv
    nombre_sin_extension = nombre_archivo.replace('.csv', '')
    
    # Dividir por guiones
    partes = nombre_sin_extension.split('-')
    
    # Omitir la primera parte (que es el nombre de la ciudad)
    # y unir el resto con espacios
    if len(partes) > 1:
        atraccion = ' '.join(partes[1:])
    else:
        atraccion = nombre_sin_extension
    
    return atraccion

In [562]:
# Función para cargar todos los datos
def cargar_datos_turisticos(ruta_data='../data'):
    """
    Carga todos los archivos CSV de las carpetas de ciudades y
    los consolida en un solo DataFrame.
    """
    dataframes = []
    
    # Obtener todas las carpetas (ciudades) en el directorio data
    carpetas_ciudades = [d for d in os.listdir(ruta_data) 
                        if os.path.isdir(os.path.join(ruta_data, d))]
    
    print(f"Ciudades encontradas: {carpetas_ciudades}")
    
    for ciudad in carpetas_ciudades:
        ruta_ciudad = os.path.join(ruta_data, ciudad)
        
        # Encontrar todos los archivos CSV en la carpeta de la ciudad
        archivos_csv = glob.glob(os.path.join(ruta_ciudad, '*.csv'))
        
        print(f"\nProcesando ciudad: {ciudad}")
        print(f"Archivos encontrados: {len(archivos_csv)}")
        
        for archivo_csv in archivos_csv:
            try:
                # Leer el archivo CSV
                df = pd.read_csv(archivo_csv)
                
                # Extraer nombre del archivo sin la ruta
                nombre_archivo = os.path.basename(archivo_csv)
                
                # Agregar columnas de ciudad y atracción
                df['ciudad'] = ciudad
                df['atraccion'] = extraer_nombre_atraccion(nombre_archivo)
                
                dataframes.append(df)
                
                print(f"  - {nombre_archivo}: {len(df)} filas")
                
            except Exception as e:
                print(f"  - Error al cargar {archivo_csv}: {e}")
    
    # Concatenar todos los DataFrames
    if dataframes:
        df_consolidado = pd.concat(dataframes, ignore_index=True)
        print(f"\n=== RESUMEN ===")
        print(f"Total de filas: {len(df_consolidado)}")
        print(f"Ciudades procesadas: {df_consolidado['ciudad'].nunique()}")
        print(f"Atracciones procesadas: {df_consolidado['atraccion'].nunique()}")
        
        return df_consolidado
    else:
        print("No se encontraron archivos CSV para procesar.")
        return pd.DataFrame()

In [563]:
# Cargar todos los datos
df_opiniones = cargar_datos_turisticos()

Ciudades encontradas: ['cancun', 'cdmx']

Procesando ciudad: cancun
Archivos encontrados: 10
  - cancun-xoximilco-cancun-by-xcaret.csv: 69 filas
  - cancun-la-isla.csv: 70 filas
  - cancun-acuario-interactivo.csv: 67 filas
  - cancun-ventura-park.csv: 72 filas
  - cancun-playa-delfines.csv: 70 filas
  - cancun-avenida-kukulkan.csv: 72 filas
  - cancun-museo-maya-de-cancun-y-zona-arqueologica-de-san-miguelito.csv: 64 filas
  - cancun-las-plazas-outlet-cancun.csv: 72 filas
  - cancun-puerto-maya-cancun.csv: 73 filas
  - cancun-playa-tortugas.csv: 71 filas

Procesando ciudad: cdmx
Archivos encontrados: 10
  - cdmx-paseo-de-la-reforma.csv: 61 filas
  - cdmx-polanco.csv: 57 filas
  - cdmx-museo-del-templo-mayor.csv: 53 filas
  - cdmx-acuario-michin-ciudad-de-mexico.csv: 74 filas
  - cdmx-mercado-de-artesanias-la-ciudadela.csv: 63 filas
  - cdmx-basilica-de-la-virgen-guadalupe.csv: 68 filas
  - cdmx-zocalo-de-la-ciudad-de-mexico.csv: 62 filas
  - cdmx-jardines-flotantes-de-xochimilco.csv: 69

In [564]:
# Conversión de tipos de datos para fechas
print("=== CONVERSIÓN DE TIPOS DE DATOS PARA FECHAS ===")

import pandas as pd
from datetime import datetime

def convertir_fecha_opinion(fecha_str):
    """
    Convierte fechas del formato '31 de agosto de 2025' a datetime
    """
    if pd.isna(fecha_str):
        return pd.NaT
    
    try:
        # Mapeo de meses en español
        meses = {
            'enero': 'January', 'febrero': 'February', 'marzo': 'March',
            'abril': 'April', 'mayo': 'May', 'junio': 'June',
            'julio': 'July', 'agosto': 'August', 'septiembre': 'September',
            'octubre': 'October', 'noviembre': 'November', 'diciembre': 'December'
        }
        
        fecha_str = str(fecha_str).strip()
        
        # Reemplazar nombres de meses en español por inglés
        for esp, eng in meses.items():
            fecha_str = fecha_str.replace(esp, eng)
        
        # Convertir a datetime
        return pd.to_datetime(fecha_str, format='%d de %B de %Y', errors='coerce')
    except:
        return pd.NaT

def convertir_fecha_estadia(fecha_str):
    """
    Convierte fechas del formato 'ago de 2025' a datetime (primer día del mes)
    """
    if pd.isna(fecha_str):
        return pd.NaT
    
    try:
        # Mapeo de meses abreviados en español
        meses = {
            'ene': 'Jan', 'feb': 'Feb', 'mar': 'Mar', 'abr': 'Apr',
            'may': 'May', 'jun': 'Jun', 'jul': 'Jul', 'ago': 'Aug',
            'sept': 'Sep', 'oct': 'Oct', 'nov': 'Nov', 'dic': 'Dec'
        }
        
        fecha_str = str(fecha_str).strip()
        
        # Reemplazar nombres de meses en español por inglés
        for esp, eng in meses.items():
            fecha_str = fecha_str.replace(esp, eng)
        
        # Agregar día 1 para hacer válida la fecha
        fecha_str = '1 ' + fecha_str
        
        # Convertir a datetime
        return pd.to_datetime(fecha_str, format='%d %b de %Y', errors='coerce')
    except:
        return pd.NaT

# Mostrar ejemplos antes de conversión
print("Ejemplos de fechas ANTES de conversión:")
print(f"FechaOpinion: {df_opiniones['FechaOpinion'].head(3).tolist()}")
print(f"FechaEstadia: {df_opiniones['FechaEstadia'].head(3).tolist()}")

# Aplicar conversiones
print("\nConvirtiendo fechas...")
df_opiniones['FechaOpinion'] = df_opiniones['FechaOpinion'].apply(convertir_fecha_opinion)
df_opiniones['FechaEstadia'] = df_opiniones['FechaEstadia'].apply(convertir_fecha_estadia)

# Mostrar ejemplos después de conversión
print("\nEjemplos de fechas DESPUÉS de conversión:")
print(f"FechaOpinion: {df_opiniones['FechaOpinion'].head(3).tolist()}")
print(f"FechaEstadia: {df_opiniones['FechaEstadia'].head(3).tolist()}")

# Mostrar tipos de datos
print(f"\nTipos de datos actualizados:")
print(f"FechaOpinion: {df_opiniones['FechaOpinion'].dtype}")
print(f"FechaEstadia: {df_opiniones['FechaEstadia'].dtype}")

# Verificar fechas nulas después de conversión
fechas_nulas_opinion = df_opiniones['FechaOpinion'].isna().sum()
fechas_nulas_estadia = df_opiniones['FechaEstadia'].isna().sum()

print(f"\nFechas que no se pudieron convertir:")
print(f"FechaOpinion: {fechas_nulas_opinion} nulas")
print(f"FechaEstadia: {fechas_nulas_estadia} nulas")

print("✅ Conversión de fechas completada")

=== CONVERSIÓN DE TIPOS DE DATOS PARA FECHAS ===
Ejemplos de fechas ANTES de conversión:
FechaOpinion: ['31 de agosto de 2025', '28 de agosto de 2025', '22 de agosto de 2025']
FechaEstadia: ['ago de 2025', 'ago de 2025', 'ago de 2025']

Convirtiendo fechas...



Ejemplos de fechas DESPUÉS de conversión:
FechaOpinion: [Timestamp('2025-08-31 00:00:00'), Timestamp('2025-08-28 00:00:00'), Timestamp('2025-08-22 00:00:00')]
FechaEstadia: [Timestamp('2025-08-01 00:00:00'), Timestamp('2025-08-01 00:00:00'), Timestamp('2025-08-01 00:00:00')]

Tipos de datos actualizados:
FechaOpinion: datetime64[ns]
FechaEstadia: datetime64[ns]

Fechas que no se pudieron convertir:
FechaOpinion: 0 nulas
FechaEstadia: 0 nulas
✅ Conversión de fechas completada


In [565]:
df_opiniones[70:85]

Titulo  \
70                                           La isla.   
71              Ricardo es excelente muy hospitalario   
72                                 Hermoso complejo!!   
73          Lindo pero no pierde el toque de tianguis   
74  Lugar para pasar un buen rato con tus seres qu...   
75  🛍️⭐⭐⭐⭐⭐ “La Isla – la joya elegante de compras...   
76                                        Estafadores   
77                Gran centro comercial al aire libre   
78                        Centro comercial muy bonito   
79    Buen lugar para ir a escapar del calor y el sol   
80                                          Mis Vacas   
81  Un bonito centro comercial al aire libre... en...   
82                          Bonito complejo comercial   
83                          La isla muy buen shopping   
84                                            Hermoso   

                                               Review  TipoViaje  \
70  Muy bonita la plaza, la experiencia en el carr...    Familia   
71  Gran servicio, servicio rápido y amable. Sin d...    Familia   
72  Muy buen diseño,y explendor!! Es un espacio al...     Pareja   
73  Muy insistentes y fastidiosos los que ofrecen ...  Solitario   
74  Este lugar es uno donde me gusta pasar tiempo ...     Amigos   
75  La Isla Shopping Village fue una gran sorpresa...     Amigos   
76  La tienda YSL está haciendo una estafa. Compré...     Pareja   
77  Qué gran lugar para visitar, aunque solo sea p...    Familia   
78  Este es un centro comercial muy agradable con ...     Pareja   
79  Vinimos aquí medio día nuestro primer día para...    Familia   
80  Excelente y con ganas de  de volver para conoc...    Familia   
81  Bonito centro comercial al aire libre con una ...     Amigos   
82  1 visita a pie de calle a este complejo, una g...     Pareja   
83  Tiene marcas como HYM, BERSKA, PULL & BEAR, Ni...    Familia   
84  Hermoso shopping que armoniza espacios de dive...     Amigos   

    Calificacion               OrigenAutor FechaOpinion FechaEstadia  ciudad  \
70             5                  1 aporte   2025-07-29   2025-07-01  cancun   
71             5                  1 aporte   2025-06-30   2025-06-01  cancun   
72             5   Buenos Aires, Argentina   2025-06-07   2025-06-01  cancun   
73             3  Ciudad de México, México   2025-05-20   2025-05-01  cancun   
74             3            Cancún, México   2025-05-18   2025-05-01  cancun   
75             5        Norte de Gales, UK   2025-05-10   2025-04-01  cancun   
76             1               Madison, MS   2025-05-05   2025-04-01  cancun   
77             5            Fort Wayne, IN   2025-04-29   2025-03-01  cancun   
78             4                     Miami   2025-04-19   2025-04-01  cancun   
79             5               Chicago, IL   2025-04-01   2025-03-01  cancun   
80             5           ALCIRA HAYDEE M   2025-03-23   2025-03-01  cancun   
81             3            Northbrook, IL   2025-03-22   2025-03-01  cancun   
82             5                 Essex, UK   2025-03-20   2025-03-01  cancun   
83             4   Buenos Aires, Argentina   2025-03-01   2025-02-01  cancun   
84             5                  Pamela L   2025-01-25   2025-01-01  cancun   

   atraccion  
70   la isla  
71   la isla  
72   la isla  
73   la isla  
74   la isla  
75   la isla  
76   la isla  
77   la isla  
78   la isla  
79   la isla  
80   la isla  
81   la isla  
82   la isla  
83   la isla  
84   la isla

In [566]:
# Análisis de la columna OrigenAutor antes de la limpieza
print("=== ANÁLISIS DE VALORES EN OrigenAutor ===")
print(f"Total de valores únicos: {df_opiniones['OrigenAutor'].nunique()}")
print(f"Total de valores no nulos: {df_opiniones['OrigenAutor'].notna().sum()}")
print(f"Valores nulos: {df_opiniones['OrigenAutor'].isna().sum()}")

print("\n=== PRIMEROS 30 VALORES ÚNICOS ===")
valores_unicos = df_opiniones['OrigenAutor'].value_counts().head(30)
for valor, count in valores_unicos.items():
    print(f"'{valor}': {count} veces")

print(f"\n=== TODOS LOS VALORES ÚNICOS (Muestra de {min(100, df_opiniones['OrigenAutor'].nunique())}) ===")
todos_valores = df_opiniones['OrigenAutor'].dropna().unique()[:100]
for i, valor in enumerate(todos_valores, 1):
    print(f"{i:2d}. '{valor}'")

=== ANÁLISIS DE VALORES EN OrigenAutor ===
Total de valores únicos: 710
Total de valores no nulos: 1325
Valores nulos: 0

=== PRIMEROS 30 VALORES ÚNICOS ===
'1 aporte': 229 veces
'Cancún, México': 42 veces
'Ciudad de México, México': 34 veces
'Buenos Aires, Argentina': 27 veces
'Bogotá, Colombia': 22 veces
'Montevideo, Uruguay': 20 veces
'Santiago, Chile': 11 veces
'Monterrey, México': 10 veces
'Estados Unidos': 10 veces
'Londres, UK': 9 veces
'Chicago, IL': 9 veces
'Nueva York, Estado de Nueva York': 9 veces
'Medellín, Colombia': 9 veces
'Madrid, España': 8 veces
'Guayaquil, Ecuador': 7 veces
'Houston, TX': 7 veces
'Lima, Perú': 7 veces
'Córdoba, Argentina': 7 veces
'Playa del Carmen, México': 7 veces
'Cali, Colombia': 7 veces
'Dallas, TX': 6 veces
'Canadá': 5 veces
'Sídney, Australia': 5 veces
'Carlos S': 5 veces
'San Diego, CA': 5 veces
'Santo Domingo, República Dominicana': 4 veces
'Mérida, México': 4 veces
'La Serena, Chile': 4 veces
'San Francisco, CA': 4 veces
'Calgary, Canadá':

In [567]:
import re

def limpiar_origen_autor(valor):
    """
    Limpia los valores de la columna OrigenAutor según los criterios especificados:
    1. Eliminar valores que contengan "aporte"
    2. Eliminar valores con más de 10 palabras
    3. Eliminar nombres propios con patrón "Nombre L" (nombre + letra)
    4. Eliminar valores en mayúsculas como "ALCIRA HAYDEE M"
    5. Mantener lugares válidos como "Puerto Rico", "Buenos Aires, Argentina"
    """
    # Si es NaN o None, mantenerlo
    if pd.isna(valor) or valor is None:
        return valor
    
    valor_str = str(valor).strip()
    
    # Criterio 1: Eliminar si contiene "aporte"
    if "aporte" in valor_str.lower():
        return None
    
    # Criterio 2: Eliminar si tiene más de 10 palabras
    palabras = valor_str.split()
    if len(palabras) > 10:
        return None
    
    # Criterio 3: Eliminar patrón "Nombre L" (nombre seguido de una sola letra)
    # Buscar patrón: palabra(s) capitalizadas seguidas de una sola letra mayúscula
    patron_nombre_letra = r'^[A-Z][a-z]+(\s+[A-Z][a-z]+)*\s+[A-Z]$'
    if re.match(patron_nombre_letra, valor_str):
        return None
    
    # Criterio 4: Eliminar si está todo en mayúsculas (nombres como "ALCIRA HAYDEE M")
    # Pero conservar lugares válidos que puedan estar en mayúsculas
    if valor_str.isupper():
        # Si es una sola palabra corta o tiene patrón de nombre personal, eliminar
        if len(palabras) <= 3 and not any(word.lower() in ['usa', 'uk', 'eu'] for word in palabras):
            return None
    
    # Criterio adicional: Eliminar si es claramente un nombre personal
    # Patrones como "María Elena F", "Ana S", etc.
    patron_nombre_personal = r'^[A-Z][a-z]+(\s+[A-Z][a-z]+)*\s+[A-Z]$'
    if re.match(patron_nombre_personal, valor_str):
        return None
    
    # Si sobrevive a todos los filtros, mantener el valor
    return valor_str

# Aplicar la función de limpieza
print("=== LIMPIEZA DE COLUMNA OrigenAutor ===")
print(f"Valores únicos antes de limpieza: {df_opiniones['OrigenAutor'].nunique()}")

# Crear columna limpia
df_opiniones['OrigenAutor_limpio'] = df_opiniones['OrigenAutor'].apply(limpiar_origen_autor)

print(f"Valores únicos después de limpieza: {df_opiniones['OrigenAutor_limpio'].nunique()}")
print(f"Valores eliminados (convertidos a None): {df_opiniones['OrigenAutor_limpio'].isna().sum() - df_opiniones['OrigenAutor'].isna().sum()}")

# Mostrar algunos ejemplos de valores eliminados
print("\n=== EJEMPLOS DE VALORES ELIMINADOS ===")
valores_eliminados = df_opiniones[
    df_opiniones['OrigenAutor'].notna() & 
    df_opiniones['OrigenAutor_limpio'].isna()
]['OrigenAutor'].value_counts().head(20)

for valor, count in valores_eliminados.items():
    print(f"'{valor}' -> ELIMINADO ({count} veces)")

print(f"\n=== VALORES CONSERVADOS (Top 20) ===")
valores_conservados = df_opiniones['OrigenAutor_limpio'].value_counts().head(20)
for valor, count in valores_conservados.items():
    print(f"'{valor}': {count} veces")

=== LIMPIEZA DE COLUMNA OrigenAutor ===
Valores únicos antes de limpieza: 710
Valores únicos después de limpieza: 458
Valores eliminados (convertidos a None): 498

=== EJEMPLOS DE VALORES ELIMINADOS ===
'1 aporte' -> ELIMINADO (229 veces)
'Carlos S' -> ELIMINADO (5 veces)
'IVAN M' -> ELIMINADO (2 veces)
'Pamela L' -> ELIMINADO (2 veces)
'Monserrat L' -> ELIMINADO (2 veces)
'Academia O' -> ELIMINADO (2 veces)
'Abaca P' -> ELIMINADO (2 veces)
'Renata M' -> ELIMINADO (2 veces)
'CARLOS ANDRES A' -> ELIMINADO (2 veces)
'Aldo N' -> ELIMINADO (2 veces)
'Acoidan L' -> ELIMINADO (2 veces)
'Vs P' -> ELIMINADO (2 veces)
'Laura C' -> ELIMINADO (2 veces)
'Aidan Macias A' -> ELIMINADO (2 veces)
'Museo emblemático sobre la historia mesoamericana. Una gran pieza de arquitectura moderna con un extraordinario diseño museográfico. A pesar de ser el museo más emblemático de las culturas indígenas de México, las autoridades del MNAH han abandonado, sin ningún mantenimiento las áreas exteriores del museo co

In [568]:
# Validación adicional de la limpieza
print("=== VALIDACIÓN DE LA LIMPIEZA ===")

# Verificar algunos casos específicos que mencionaste
casos_test = ['Pamela L', 'Cifuentes E', 'ALCIRA HAYDEE M', '1 aporte', 
              'Puerto Rico', 'Buenos Aires, Argentina', 'San Juan, Puerto Rico',
              'Ciudad de México, México']

print("Prueba de casos específicos:")
for caso in casos_test:
    resultado = limpiar_origen_autor(caso)
    estado = "✅ CONSERVADO" if resultado is not None else "❌ ELIMINADO"
    print(f"'{caso}' -> {estado}")

print(f"\n=== ANÁLISIS FINAL ===")
print(f"Registros totales: {len(df_opiniones)}")
print(f"OrigenAutor original - Valores únicos: {df_opiniones['OrigenAutor'].nunique()}")
print(f"OrigenAutor original - Valores no nulos: {df_opiniones['OrigenAutor'].notna().sum()}")
print(f"OrigenAutor limpio - Valores únicos: {df_opiniones['OrigenAutor_limpio'].nunique()}")
print(f"OrigenAutor limpio - Valores no nulos: {df_opiniones['OrigenAutor_limpio'].notna().sum()}")

# Reemplazar la columna original con la limpia
print(f"\n=== REEMPLAZANDO COLUMNA ORIGINAL ===")
df_opiniones['OrigenAutor'] = df_opiniones['OrigenAutor_limpio']
df_opiniones.drop('OrigenAutor_limpio', axis=1, inplace=True)

print("✅ Columna OrigenAutor limpiada exitosamente")
print(f"Valores únicos finales en OrigenAutor: {df_opiniones['OrigenAutor'].nunique()}")

# Mostrar muestra de los valores finales más comunes
print(f"\n=== TOP 15 PAÍSES/LUGARES MÁS COMUNES (DESPUÉS DE LIMPIEZA) ===")
valores_finales = df_opiniones['OrigenAutor'].value_counts().head(15)
for valor, count in valores_finales.items():
    print(f"'{valor}': {count} opiniones")

=== VALIDACIÓN DE LA LIMPIEZA ===
Prueba de casos específicos:
'Pamela L' -> ❌ ELIMINADO
'Cifuentes E' -> ❌ ELIMINADO
'ALCIRA HAYDEE M' -> ❌ ELIMINADO
'1 aporte' -> ❌ ELIMINADO
'Puerto Rico' -> ✅ CONSERVADO
'Buenos Aires, Argentina' -> ✅ CONSERVADO
'San Juan, Puerto Rico' -> ✅ CONSERVADO
'Ciudad de México, México' -> ✅ CONSERVADO

=== ANÁLISIS FINAL ===
Registros totales: 1325
OrigenAutor original - Valores únicos: 710
OrigenAutor original - Valores no nulos: 1325
OrigenAutor limpio - Valores únicos: 458
OrigenAutor limpio - Valores no nulos: 827

=== REEMPLAZANDO COLUMNA ORIGINAL ===
✅ Columna OrigenAutor limpiada exitosamente
Valores únicos finales en OrigenAutor: 458

=== TOP 15 PAÍSES/LUGARES MÁS COMUNES (DESPUÉS DE LIMPIEZA) ===
'Cancún, México': 42 opiniones
'Ciudad de México, México': 34 opiniones
'Buenos Aires, Argentina': 27 opiniones
'Bogotá, Colombia': 22 opiniones
'Montevideo, Uruguay': 20 opiniones
'Santiago, Chile': 11 opiniones
'Monterrey, México': 10 opiniones
'Estados 

In [569]:
df_opiniones[70:85]

Titulo  \
70                                           La isla.   
71              Ricardo es excelente muy hospitalario   
72                                 Hermoso complejo!!   
73          Lindo pero no pierde el toque de tianguis   
74  Lugar para pasar un buen rato con tus seres qu...   
75  🛍️⭐⭐⭐⭐⭐ “La Isla – la joya elegante de compras...   
76                                        Estafadores   
77                Gran centro comercial al aire libre   
78                        Centro comercial muy bonito   
79    Buen lugar para ir a escapar del calor y el sol   
80                                          Mis Vacas   
81  Un bonito centro comercial al aire libre... en...   
82                          Bonito complejo comercial   
83                          La isla muy buen shopping   
84                                            Hermoso   

                                               Review  TipoViaje  \
70  Muy bonita la plaza, la experiencia en el carr...    Familia   
71  Gran servicio, servicio rápido y amable. Sin d...    Familia   
72  Muy buen diseño,y explendor!! Es un espacio al...     Pareja   
73  Muy insistentes y fastidiosos los que ofrecen ...  Solitario   
74  Este lugar es uno donde me gusta pasar tiempo ...     Amigos   
75  La Isla Shopping Village fue una gran sorpresa...     Amigos   
76  La tienda YSL está haciendo una estafa. Compré...     Pareja   
77  Qué gran lugar para visitar, aunque solo sea p...    Familia   
78  Este es un centro comercial muy agradable con ...     Pareja   
79  Vinimos aquí medio día nuestro primer día para...    Familia   
80  Excelente y con ganas de  de volver para conoc...    Familia   
81  Bonito centro comercial al aire libre con una ...     Amigos   
82  1 visita a pie de calle a este complejo, una g...     Pareja   
83  Tiene marcas como HYM, BERSKA, PULL & BEAR, Ni...    Familia   
84  Hermoso shopping que armoniza espacios de dive...     Amigos   

    Calificacion               OrigenAutor FechaOpinion FechaEstadia  ciudad  \
70             5                      None   2025-07-29   2025-07-01  cancun   
71             5                      None   2025-06-30   2025-06-01  cancun   
72             5   Buenos Aires, Argentina   2025-06-07   2025-06-01  cancun   
73             3  Ciudad de México, México   2025-05-20   2025-05-01  cancun   
74             3            Cancún, México   2025-05-18   2025-05-01  cancun   
75             5        Norte de Gales, UK   2025-05-10   2025-04-01  cancun   
76             1               Madison, MS   2025-05-05   2025-04-01  cancun   
77             5            Fort Wayne, IN   2025-04-29   2025-03-01  cancun   
78             4                     Miami   2025-04-19   2025-04-01  cancun   
79             5               Chicago, IL   2025-04-01   2025-03-01  cancun   
80             5                      None   2025-03-23   2025-03-01  cancun   
81             3            Northbrook, IL   2025-03-22   2025-03-01  cancun   
82             5                 Essex, UK   2025-03-20   2025-03-01  cancun   
83             4   Buenos Aires, Argentina   2025-03-01   2025-02-01  cancun   
84             5                      None   2025-01-25   2025-01-01  cancun   

   atraccion  
70   la isla  
71   la isla  
72   la isla  
73   la isla  
74   la isla  
75   la isla  
76   la isla  
77   la isla  
78   la isla  
79   la isla  
80   la isla  
81   la isla  
82   la isla  
83   la isla  
84   la isla

In [570]:
# Crear una copia del dataset para análisis (preservar el original)
df_analisis = df_opiniones.copy()

print(f"Dataset original preservado: {len(df_opiniones)} filas")
print(f"Dataset para análisis creado: {len(df_analisis)} filas")
print("✅ Se trabajará con 'df_analisis' para no modificar el dataset original")

Dataset original preservado: 1325 filas
Dataset para análisis creado: 1325 filas
✅ Se trabajará con 'df_analisis' para no modificar el dataset original


## 2. Análisis Básico de Calidad de Datos

In [571]:
# Información general del dataset
print("=== INFORMACIÓN GENERAL DEL DATASET ===")
print(f"Dimensiones: {df_analisis.shape}")
print(f"Número de filas: {df_analisis.shape[0]:,}")
print(f"Número de columnas: {df_analisis.shape[1]}")
print("\nColumnas del dataset:")
for i, col in enumerate(df_analisis.columns, 1):
    print(f"{i}. {col}")

=== INFORMACIÓN GENERAL DEL DATASET ===
Dimensiones: (1325, 9)
Número de filas: 1,325
Número de columnas: 9

Columnas del dataset:
1. Titulo
2. Review
3. TipoViaje
4. Calificacion
5. OrigenAutor
6. FechaOpinion
7. FechaEstadia
8. ciudad
9. atraccion


In [572]:
# Información detallada de tipos de datos
print("=== TIPOS DE DATOS ===")
df_analisis.info()

=== TIPOS DE DATOS ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1325 entries, 0 to 1324
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Titulo        1317 non-null   object        
 1   Review        1325 non-null   object        
 2   TipoViaje     1121 non-null   object        
 3   Calificacion  1325 non-null   int64         
 4   OrigenAutor   827 non-null    object        
 5   FechaOpinion  1325 non-null   datetime64[ns]
 6   FechaEstadia  1325 non-null   datetime64[ns]
 7   ciudad        1325 non-null   object        
 8   atraccion     1325 non-null   object        
dtypes: datetime64[ns](2), int64(1), object(6)
memory usage: 93.3+ KB


In [573]:
# Completar valores nulos en el dataset original con valores descriptivos
print("=== COMPLETANDO VALORES NULOS EN DATASET ORIGINAL ===")

# Mostrar valores nulos antes de la limpieza
print("Valores nulos ANTES de completar:")
print(f"- Titulo: {df_opiniones['Titulo'].isna().sum()} nulos")
print(f"- TipoViaje: {df_opiniones['TipoViaje'].isna().sum()} nulos") 
print(f"- OrigenAutor: {df_opiniones['OrigenAutor'].isna().sum()} nulos")

# Completar valores nulos con valores descriptivos
df_opiniones['Titulo'].fillna('sin titulo', inplace=True)
df_opiniones['TipoViaje'].fillna('desconocido', inplace=True)
df_opiniones['OrigenAutor'].fillna('anonimo', inplace=True)

# Mostrar valores nulos después de la limpieza
print("\nValores nulos DESPUÉS de completar:")
print(f"- Titulo: {df_opiniones['Titulo'].isna().sum()} nulos")
print(f"- TipoViaje: {df_opiniones['TipoViaje'].isna().sum()} nulos")
print(f"- OrigenAutor: {df_opiniones['OrigenAutor'].isna().sum()} nulos")

print("\n✅ Valores nulos completados exitosamente en dataset original")

# Mostrar algunos ejemplos de los valores agregados
print(f"\n=== DISTRIBUCIÓN DE VALORES AGREGADOS ===")
print(f"'sin titulo': {(df_opiniones['Titulo'] == 'sin titulo').sum()} registros")
print(f"'desconocido': {(df_opiniones['TipoViaje'] == 'desconocido').sum()} registros")
print(f"'anonimo': {(df_opiniones['OrigenAutor'] == 'anonimo').sum()} registros")

=== COMPLETANDO VALORES NULOS EN DATASET ORIGINAL ===
Valores nulos ANTES de completar:
- Titulo: 8 nulos
- TipoViaje: 204 nulos
- OrigenAutor: 498 nulos

Valores nulos DESPUÉS de completar:
- Titulo: 0 nulos
- TipoViaje: 0 nulos
- OrigenAutor: 0 nulos

✅ Valores nulos completados exitosamente en dataset original

=== DISTRIBUCIÓN DE VALORES AGREGADOS ===
'sin titulo': 8 registros
'desconocido': 204 registros
'anonimo': 498 registros


/tmp/ipykernel_34340/1473809398.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_opiniones['Titulo'].fillna('sin titulo', inplace=True)
/tmp/ipykernel_34340/1473809398.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace

In [574]:
# Actualizar el dataset de análisis con los cambios del dataset original
df_analisis = df_opiniones.copy()

print("=== VERIFICACIÓN FINAL DE VALORES NULOS ===")
print("Dataset actualizado para análisis")
print(f"Dimensiones: {df_analisis.shape}")

# Verificar que no hay valores nulos en las columnas principales
valores_nulos_final = df_analisis.isnull().sum()
print(f"\nValores nulos por columna:")
for columna, nulos in valores_nulos_final.items():
    if nulos > 0:
        print(f"- {columna}: {nulos} nulos ⚠️")
    else:
        print(f"- {columna}: {nulos} nulos ✅")

print(f"\n=== RESUMEN DE LIMPIEZA COMPLETADA ===")
print("✅ Limpieza de OrigenAutor: Eliminados nombres personales, conservados lugares")
print("✅ Completado de valores nulos:")
print("   - Títulos faltantes → 'sin titulo'")
print("   - Tipos de viaje faltantes → 'desconocido'") 
print("   - Origen de autor faltante → 'anonimo'")
print("✅ Dataset listo para análisis sin pérdida de registros")

=== VERIFICACIÓN FINAL DE VALORES NULOS ===
Dataset actualizado para análisis
Dimensiones: (1325, 9)

Valores nulos por columna:
- Titulo: 0 nulos ✅
- Review: 0 nulos ✅
- TipoViaje: 0 nulos ✅
- Calificacion: 0 nulos ✅
- OrigenAutor: 0 nulos ✅
- FechaOpinion: 0 nulos ✅
- FechaEstadia: 0 nulos ✅
- ciudad: 0 nulos ✅
- atraccion: 0 nulos ✅

=== RESUMEN DE LIMPIEZA COMPLETADA ===
✅ Limpieza de OrigenAutor: Eliminados nombres personales, conservados lugares
✅ Completado de valores nulos:
   - Títulos faltantes → 'sin titulo'
   - Tipos de viaje faltantes → 'desconocido'
   - Origen de autor faltante → 'anonimo'
✅ Dataset listo para análisis sin pérdida de registros


In [575]:
# Análisis de valores nulos
print("=== ANÁLISIS DE VALORES NULOS ===")
valores_nulos = df_analisis.isnull().sum()
porcentaje_nulos = (valores_nulos / len(df_analisis)) * 100

resumen_nulos = pd.DataFrame({
    'Valores_Nulos': valores_nulos,
    'Porcentaje': porcentaje_nulos
})

resumen_nulos = resumen_nulos.sort_values('Porcentaje', ascending=False)
print(resumen_nulos)

=== ANÁLISIS DE VALORES NULOS ===
              Valores_Nulos  Porcentaje
Titulo                    0         0.0
Review                    0         0.0
TipoViaje                 0         0.0
Calificacion              0         0.0
OrigenAutor               0         0.0
FechaOpinion              0         0.0
FechaEstadia              0         0.0
ciudad                    0         0.0
atraccion                 0         0.0


In [576]:
# Análisis y eliminación de duplicados
print("=== ANÁLISIS Y ELIMINACIÓN DE DUPLICADOS ===")

# Guardar dimensiones antes de eliminar duplicados
filas_antes = len(df_analisis)

# Duplicados completos
duplicados_completos = df_analisis.duplicated().sum()
print(f"Filas completamente duplicadas encontradas: {duplicados_completos}")

# Duplicados por combinación de columnas importantes
if 'Titulo' in df_analisis.columns and 'Review' in df_analisis.columns:
    duplicados_contenido = df_analisis.duplicated(subset=['Titulo', 'Review', 'ciudad', 'atraccion']).sum()
    print(f"Duplicados por título + review + ciudad + atracción: {duplicados_contenido}")

# Eliminar duplicados completos
if duplicados_completos > 0:
    porcentaje_duplicados = (duplicados_completos / len(df_analisis)) * 100
    print(f"Porcentaje de duplicados completos: {porcentaje_duplicados:.2f}%")
    
    print(f"\n🔄 Eliminando {duplicados_completos} filas duplicadas...")
    df_analisis = df_analisis.drop_duplicates()
    
    # Aplicar la misma eliminación al dataset original
    df_opiniones = df_opiniones.drop_duplicates()
    
    filas_despues = len(df_analisis)
    filas_eliminadas = filas_antes - filas_despues
    
    print(f"✅ Duplicados eliminados exitosamente")
    print(f"   Filas antes: {filas_antes:,}")
    print(f"   Filas después: {filas_despues:,}")
    print(f"   Filas eliminadas: {filas_eliminadas:,}")
else:
    print("✅ No se encontraron duplicados completos para eliminar")

=== ANÁLISIS Y ELIMINACIÓN DE DUPLICADOS ===
Filas completamente duplicadas encontradas: 11
Duplicados por título + review + ciudad + atracción: 11
Porcentaje de duplicados completos: 0.83%

🔄 Eliminando 11 filas duplicadas...
✅ Duplicados eliminados exitosamente
   Filas antes: 1,325
   Filas después: 1,314
   Filas eliminadas: 11
✅ Duplicados eliminados exitosamente
   Filas antes: 1,325
   Filas después: 1,314
   Filas eliminadas: 11


In [577]:
# Análisis avanzado: Detectar contenidos mal ubicados entre columnas
print("=== ANÁLISIS AVANZADO DE CONTENIDOS MAL UBICADOS ===")
print("Detectando cuando el contenido de una columna aparece en otra columna diferente...")

def detectar_contenidos_mal_ubicados(dataframe, min_longitud=10):
    """
    Detecta cuando el contenido de una columna aparece incorrectamente en otra columna.
    
    Args:
        dataframe: DataFrame a analizar
        min_longitud: Longitud mínima del texto para considerar en el análisis
    
    Returns:
        dict: Resultados de contenidos mal ubicados por par de columnas
    """
    
    # Solo columnas de texto para analizar (excluyendo fechas que son datetime)
    columnas_texto = ['Titulo', 'Review', 'TipoViaje', 'OrigenAutor']
    
    # Filtrar solo las columnas que existen en el dataframe y son de tipo texto
    columnas_existentes = []
    for col in columnas_texto:
        if col in dataframe.columns:
            # Verificar que la columna sea de tipo texto o pueda convertirse a texto
            dtype = dataframe[col].dtype
            if dtype == 'object' or pd.api.types.is_string_dtype(dtype):
                columnas_existentes.append(col)
    
    resultados = {}
    problemas_encontrados = []
    
    print(f"Analizando columnas de texto: {columnas_existentes}")
    print(f"Longitud mínima de texto: {min_longitud} caracteres\n")
    
    # Iterar sobre todas las combinaciones de columnas
    for i, col1 in enumerate(columnas_existentes):
        for j, col2 in enumerate(columnas_existentes):
            if i != j:  # No comparar una columna consigo misma
                
                # Crear sets de valores únicos no nulos y con longitud suficiente
                valores_col1 = set()
                valores_col2 = set()
                
                for valor in dataframe[col1].dropna():
                    valor_str = str(valor).strip()
                    if len(valor_str) >= min_longitud:
                        valores_col1.add(valor_str.lower())
                
                for valor in dataframe[col2].dropna():
                    valor_str = str(valor).strip()
                    if len(valor_str) >= min_longitud:
                        valores_col2.add(valor_str.lower())
                
                # Encontrar intersección (valores que aparecen en ambas columnas)
                contenidos_duplicados = valores_col1.intersection(valores_col2)
                
                if contenidos_duplicados:
                    par_columnas = f"{col1} ↔ {col2}"
                    resultados[par_columnas] = contenidos_duplicados
                    
                    print(f"⚠️  PROBLEMA DETECTADO: {par_columnas}")
                    print(f"   Contenidos duplicados encontrados: {len(contenidos_duplicados)}")
                    
                    # Mostrar algunos ejemplos
                    ejemplos = list(contenidos_duplicados)[:3]
                    for ejemplo in ejemplos:
                        # Encontrar las filas específicas donde ocurre esto usando comparación segura
                        # Convertir columnas a string antes de usar .str accessor
                        col1_str = dataframe[col1].astype(str).str.lower().str.strip()
                        col2_str = dataframe[col2].astype(str).str.lower().str.strip()
                        
                        filas_col1 = dataframe[col1_str == ejemplo].index.tolist()
                        filas_col2 = dataframe[col2_str == ejemplo].index.tolist()
                        
                        print(f"   📝 Ejemplo: '{ejemplo[:50]}...'")
                        print(f"      - En {col1}: filas {filas_col1[:3]}")
                        print(f"      - En {col2}: filas {filas_col2[:3]}")
                    
                    print()
                    
                    # Guardar información del problema
                    problemas_encontrados.append({
                        'columna_1': col1,
                        'columna_2': col2,
                        'contenidos_duplicados': len(contenidos_duplicados),
                        'ejemplos': ejemplos[:5]
                    })
    
    return resultados, problemas_encontrados

# Ejecutar el análisis
contenidos_mal_ubicados, problemas = detectar_contenidos_mal_ubicados(df_analisis)

print(f"=== RESUMEN DEL ANÁLISIS ===")
if problemas:
    print(f"❌ Se encontraron {len(problemas)} problemas de contenidos mal ubicados")
    print(f"Total de pares de columnas con problemas: {len(contenidos_mal_ubicados)}")
    
    # Resumen por tipo de problema
    print(f"\n📊 ESTADÍSTICAS DE PROBLEMAS:")
    for problema in problemas:
        print(f"   {problema['columna_1']} ↔ {problema['columna_2']}: {problema['contenidos_duplicados']} contenidos duplicados")
        
else:
    print("✅ No se encontraron contenidos mal ubicados entre columnas")
    print("   El dataset parece estar bien estructurado en este aspecto")

=== ANÁLISIS AVANZADO DE CONTENIDOS MAL UBICADOS ===
Detectando cuando el contenido de una columna aparece en otra columna diferente...
Analizando columnas de texto: ['Titulo', 'Review', 'TipoViaje', 'OrigenAutor']
Longitud mínima de texto: 10 caracteres

⚠️  PROBLEMA DETECTADO: Titulo ↔ OrigenAutor
   Contenidos duplicados encontrados: 21
   📝 Ejemplo: 'mercado de artesanias la ciudadela, ciudad de méxi...'
      - En Titulo: filas [964]
      - En OrigenAutor: filas [964]
   📝 Ejemplo: 'ella es la que hace a méxico, méxico....'
      - En Titulo: filas [1063]
      - En OrigenAutor: filas [1063]
   📝 Ejemplo: 'un palacio de ensueño dentro de la ciudad de méxic...'
      - En Titulo: filas [1247]
      - En OrigenAutor: filas [1247]

⚠️  PROBLEMA DETECTADO: Titulo ↔ OrigenAutor
   Contenidos duplicados encontrados: 21
   📝 Ejemplo: 'mercado de artesanias la ciudadela, ciudad de méxi...'
      - En Titulo: filas [964]
      - En OrigenAutor: filas [964]
   📝 Ejemplo: 'ella es la que ha

In [578]:
# Examinar en detalle los casos problemáticos y corregirlos
print("=== ANÁLISIS DETALLADO DE CASOS PROBLEMÁTICOS ===")

def examinar_y_corregir_contenidos_mal_ubicados(dataframe):
    """
    Examina casos específicos donde Titulo y OrigenAutor tienen el mismo contenido
    y corrige automáticamente cuando es posible detectar el error.
    """
    
    print("Examinando registros con contenido duplicado entre Titulo y OrigenAutor...\n")
    
    casos_problematicos = []
    correcciones_realizadas = 0
    
    # Encontrar todas las filas donde Titulo y OrigenAutor son idénticos
    for idx, row in dataframe.iterrows():
        titulo = str(row['Titulo']).strip()
        origen = str(row['OrigenAutor']).strip()
        
        # Si son idénticos y tienen longitud significativa
        if titulo.lower() == origen.lower() and len(titulo) > 10:
            
            casos_problematicos.append({
                'fila': idx,
                'titulo': titulo,
                'origen': origen,
                'ciudad': row['ciudad'],
                'atraccion': row['atraccion']
            })
            
            print(f"🔍 FILA {idx}:")
            print(f"   Titulo: '{titulo[:80]}...'")
            print(f"   OrigenAutor: '{origen[:80]}...'")
            print(f"   Ciudad: {row['ciudad']}")
            print(f"   Atracción: {row['atraccion']}")
            
            # Lógica de corrección automática

            dataframe.at[idx, 'OrigenAutor'] = 'anonimo'
            correcciones_realizadas += 1
            print(f"   ✅ CORREGIDO: OrigenAutor cambiado a 'anonimo'")
            

            
            print()
    
    return casos_problematicos, correcciones_realizadas

# Ejecutar el análisis y corrección
casos_problematicos, correcciones = examinar_y_corregir_contenidos_mal_ubicados(df_analisis)

print(f"=== RESUMEN DE CORRECCIONES ===")
print(f"📊 Casos problemáticos encontrados: {len(casos_problematicos)}")
print(f"✅ Correcciones automáticas realizadas: {correcciones}")
print(f"⚠️  Casos que requieren revisión manual: {len(casos_problematicos) - correcciones}")

if correcciones > 0:
    print(f"\n🔄 Actualizando dataset original...")
    # Aplicar las mismas correcciones al dataset original
    df_opiniones.update(df_analisis)

=== ANÁLISIS DETALLADO DE CASOS PROBLEMÁTICOS ===
Examinando registros con contenido duplicado entre Titulo y OrigenAutor...

🔍 FILA 282:
   Titulo: 'Playa imperdible de México...'
   OrigenAutor: 'Playa imperdible de México...'
   Ciudad: cancun
   Atracción: playa delfines
   ✅ CORREGIDO: OrigenAutor cambiado a 'anonimo'

🔍 FILA 705:
   Titulo: 'Paseo de la Reforma, Ciudad de México en octubre...'
   OrigenAutor: 'Paseo de la Reforma, Ciudad de México en octubre...'
   Ciudad: cdmx
   Atracción: paseo de la reforma
   ✅ CORREGIDO: OrigenAutor cambiado a 'anonimo'

🔍 FILA 767:
   Titulo: 'Colonia exclusiva de la Ciudad de México...'
   OrigenAutor: 'Colonia exclusiva de la Ciudad de México...'
   Ciudad: cdmx
   Atracción: polanco
   ✅ CORREGIDO: OrigenAutor cambiado a 'anonimo'

🔍 FILA 801:
   Titulo: 'El Beverly Hills de la ciudad de México...'
   OrigenAutor: 'El Beverly Hills de la ciudad de México...'
   Ciudad: cdmx
   Atracción: polanco
   ✅ CORREGIDO: OrigenAutor cambiado a 'a

In [579]:
# Verificación final: Re-ejecutar análisis después de correcciones
print("=== VERIFICACIÓN POST-CORRECCIONES ===")
print("Re-ejecutando análisis de contenidos mal ubicados después de las correcciones...\n")

# Volver a ejecutar el análisis de contenidos mal ubicados
contenidos_mal_ubicados_final, problemas_final = detectar_contenidos_mal_ubicados(df_analisis, min_longitud=10)

print(f"\n=== COMPARACIÓN ANTES vs DESPUÉS ===")
print(f"📊 ANTES DE CORRECCIONES:")
print(f"   - Problemas detectados: {len(problemas)}")
print(f"   - Contenidos duplicados: {sum(p['contenidos_duplicados'] for p in problemas)}")

print(f"\n📊 DESPUÉS DE CORRECCIONES:")
if problemas_final:
    print(f"   - Problemas detectados: {len(problemas_final)}")
    print(f"   - Contenidos duplicados: {sum(p['contenidos_duplicados'] for p in problemas_final)}")
else:
    print("   - ✅ No se detectaron más problemas de contenidos mal ubicados")

# Verificar algunos ejemplos de las correcciones aplicadas
print(f"\n=== EJEMPLOS DE CORRECCIONES APLICADAS ===")
ejemplos_corregidos = df_analisis[
    (df_analisis['OrigenAutor'] == 'anonimo') | 
    (df_analisis['Titulo'] == 'sin titulo')
].head(10)

if len(ejemplos_corregidos) > 0:
    print("Muestra de registros corregidos:")
    for idx, row in ejemplos_corregidos.iterrows():
        print(f"Fila {idx}: Titulo='{row['Titulo'][:30]}...', OrigenAutor='{row['OrigenAutor']}'")
else:
    print("No se encontraron registros con correcciones aplicadas en la muestra")

print(f"\n✅ ANÁLISIS DE CONTENIDOS MAL UBICADOS COMPLETADO")
print("Dataset mejorado y listo para análisis posteriores")

=== VERIFICACIÓN POST-CORRECCIONES ===
Re-ejecutando análisis de contenidos mal ubicados después de las correcciones...

Analizando columnas de texto: ['Titulo', 'Review', 'TipoViaje', 'OrigenAutor']
Longitud mínima de texto: 10 caracteres


=== COMPARACIÓN ANTES vs DESPUÉS ===
📊 ANTES DE CORRECCIONES:
   - Problemas detectados: 2
   - Contenidos duplicados: 42

📊 DESPUÉS DE CORRECCIONES:
   - ✅ No se detectaron más problemas de contenidos mal ubicados

=== EJEMPLOS DE CORRECCIONES APLICADAS ===
Muestra de registros corregidos:
Fila 0: Titulo='Divertido y emocionante...', OrigenAutor='anonimo'
Fila 1: Titulo='Increíble...', OrigenAutor='anonimo'
Fila 2: Titulo='Xochimilco excelente experienc...', OrigenAutor='anonimo'
Fila 3: Titulo='Xoximilco increible experienci...', OrigenAutor='anonimo'
Fila 5: Titulo='Cumpleaños especial!...', OrigenAutor='anonimo'
Fila 7: Titulo='Xoximilco Experience...', OrigenAutor='anonimo'
Fila 8: Titulo='Genial....', OrigenAutor='anonimo'
Fila 9: Titulo='Noc

## 3. Análisis Descriptivo por Categorías

In [580]:
# Distribución por ciudades
print("=== DISTRIBUCIÓN POR CIUDADES ===")
distribucion_ciudades = df_analisis['ciudad'].value_counts()
print(distribucion_ciudades)
print(f"\nPorcentaje por ciudad:")
print((distribucion_ciudades / distribucion_ciudades.sum() * 100).round(2))

=== DISTRIBUCIÓN POR CIUDADES ===
ciudad
cancun    700
cdmx      614
Name: count, dtype: int64

Porcentaje por ciudad:
ciudad
cancun    53.27
cdmx      46.73
Name: count, dtype: float64


In [581]:
# Distribución por atracciones (top 10)
print("=== TOP 10 ATRACCIONES CON MÁS OPINIONES ===")
distribucion_atracciones = df_analisis['atraccion'].value_counts().head(10)
print(distribucion_atracciones)

=== TOP 10 ATRACCIONES CON MÁS OPINIONES ===
atraccion
acuario michin ciudad de mexico     74
puerto maya cancun                  73
ventura park                        72
las plazas outlet cancun            72
avenida kukulkan                    72
playa tortugas                      71
la isla                             70
playa delfines                      70
jardines flotantes de xochimilco    69
xoximilco cancun by xcaret          69
Name: count, dtype: int64


In [582]:
# Análisis de calificaciones
if 'Calificacion' in df_analisis.columns:
    print("=== ANÁLISIS DE CALIFICACIONES ===")
    
    # Convertir calificaciones a numérico si no lo están
    df_analisis['Calificacion'] = pd.to_numeric(df_analisis['Calificacion'], errors='coerce')
    
    print("Estadísticas descriptivas de calificaciones:")
    print(df_analisis['Calificacion'].describe())
    
    print("\nDistribución de calificaciones:")
    print(df_analisis['Calificacion'].value_counts().sort_index())
    
    print(f"\nCalificación promedio general: {df_analisis['Calificacion'].mean():.2f}")

=== ANÁLISIS DE CALIFICACIONES ===
Estadísticas descriptivas de calificaciones:
count    1314.000000
mean        4.259513
std         1.131999
min         1.000000
25%         4.000000
50%         5.000000
75%         5.000000
max         5.000000
Name: Calificacion, dtype: float64

Distribución de calificaciones:
Calificacion
1     78
2     34
3    145
4    269
5    788
Name: count, dtype: int64

Calificación promedio general: 4.26
count    1314.000000
mean        4.259513
std         1.131999
min         1.000000
25%         4.000000
50%         5.000000
75%         5.000000
max         5.000000
Name: Calificacion, dtype: float64

Distribución de calificaciones:
Calificacion
1     78
2     34
3    145
4    269
5    788
Name: count, dtype: int64

Calificación promedio general: 4.26


In [583]:
# Análisis de tipos de viaje
if 'TipoViaje' in df_analisis.columns:
    print("=== ANÁLISIS DE TIPOS DE VIAJE ===")
    distribucion_tipos = df_analisis['TipoViaje'].value_counts()
    print(distribucion_tipos)
    print(f"\nPorcentaje por tipo de viaje:")
    print((distribucion_tipos / distribucion_tipos.sum() * 100).round(2))

=== ANÁLISIS DE TIPOS DE VIAJE ===
TipoViaje
Familia        471
Pareja         338
desconocido    203
Amigos         175
Solitario      115
Negocios        12
Name: count, dtype: int64

Porcentaje por tipo de viaje:
TipoViaje
Familia        35.84
Pareja         25.72
desconocido    15.45
Amigos         13.32
Solitario       8.75
Negocios        0.91
Name: count, dtype: float64


## 4. Análisis de Longitud de Textos

In [584]:
# Análisis de longitud de títulos y reviews
if 'Titulo' in df_analisis.columns:
    df_analisis['longitud_titulo'] = df_analisis['Titulo'].astype(str).str.len()
    print("=== ANÁLISIS DE LONGITUD DE TÍTULOS ===")
    print(df_analisis['longitud_titulo'].describe())

if 'Review' in df_analisis.columns:
    df_analisis['longitud_review'] = df_analisis['Review'].astype(str).str.len()
    print("\n=== ANÁLISIS DE LONGITUD DE REVIEWS ===")
    print(df_analisis['longitud_review'].describe())
    
    # Contar palabras en reviews
    df_analisis['palabras_review'] = df_analisis['Review'].astype(str).str.split().str.len()
    print("\n=== NÚMERO DE PALABRAS EN REVIEWS ===")
    print(df_analisis['palabras_review'].describe())

=== ANÁLISIS DE LONGITUD DE TÍTULOS ===
count    1314.000000
mean       27.614155
std        19.994923
min         2.000000
25%        14.000000
50%        22.000000
75%        35.000000
max       129.000000
Name: longitud_titulo, dtype: float64

=== ANÁLISIS DE LONGITUD DE REVIEWS ===
count    1314.000000
mean      312.919330
std       276.529729
min       100.000000
25%       145.000000
50%       203.000000
75%       366.750000
max      2162.000000
Name: longitud_review, dtype: float64

=== NÚMERO DE PALABRAS EN REVIEWS ===
count    1314.000000
mean       53.611872
std        47.721281
min        11.000000
25%        25.000000
50%        35.000000
75%        63.000000
max       361.000000
Name: palabras_review, dtype: float64


## 5. Análisis Temporal

In [585]:
# Análisis temporal de opiniones
if 'FechaOpinion' in df_analisis.columns:
    print("=== ANÁLISIS TEMPORAL DE OPINIONES ===")
    
    # Mostrar algunas fechas de ejemplo para entender el formato
    print("Ejemplos de fechas en el dataset:")
    print(df_analisis['FechaOpinion'].head(10).tolist())
    
    # Contar valores únicos en fechas
    print(f"\nFechas únicas de opinión: {df_analisis['FechaOpinion'].nunique()}")
    
    # Mostrar las fechas más comunes
    print("\nTop 10 fechas con más opiniones:")
    print(df_analisis['FechaOpinion'].value_counts().head(10))

=== ANÁLISIS TEMPORAL DE OPINIONES ===
Ejemplos de fechas en el dataset:
[Timestamp('2025-08-31 00:00:00'), Timestamp('2025-08-28 00:00:00'), Timestamp('2025-08-22 00:00:00'), Timestamp('2025-08-21 00:00:00'), Timestamp('2025-08-19 00:00:00'), Timestamp('2025-08-19 00:00:00'), Timestamp('2025-08-17 00:00:00'), Timestamp('2025-08-12 00:00:00'), Timestamp('2025-08-12 00:00:00'), Timestamp('2025-08-09 00:00:00')]

Fechas únicas de opinión: 752

Top 10 fechas con más opiniones:
FechaOpinion
2024-07-22    10
2024-09-12    10
2024-09-04     9
2024-09-11     8
2025-08-14     8
2025-07-24     8
2025-08-31     7
2025-07-15     7
2025-06-10     6
2024-08-19     6
Name: count, dtype: int64


## 6. Resumen Ejecutivo

In [586]:
# Resumen ejecutivo del análisis
print("\n" + "="*60)
print("                    RESUMEN EJECUTIVO")
print("="*60)

print(f"📊 VOLUMEN DE DATOS:")
print(f"   • Total de opiniones: {len(df_analisis):,}")
print(f"   • Ciudades analizadas: {df_analisis['ciudad'].nunique()}")
print(f"   • Atracciones totales: {df_analisis['atraccion'].nunique()}")

print(f"\n🏙️ DISTRIBUCIÓN POR CIUDADES:")
for ciudad, cantidad in df_analisis['ciudad'].value_counts().items():
    porcentaje = (cantidad / len(df_analisis)) * 100
    print(f"   • {ciudad.upper()}: {cantidad:,} opiniones ({porcentaje:.1f}%)")

if 'Calificacion' in df_analisis.columns:
    print(f"\n⭐ CALIFICACIONES:")
    print(f"   • Promedio general: {df_analisis['Calificacion'].mean():.2f}/5")
    print(f"   • Mediana: {df_analisis['Calificacion'].median():.1f}/5")

if 'TipoViaje' in df_analisis.columns:
    tipo_mas_comun = df_analisis['TipoViaje'].mode()[0]
    print(f"\n👥 TIPO DE VIAJE:")
    print(f"   • Más común: {tipo_mas_comun}")

print(f"\n🔍 CALIDAD DE DATOS:")
print(f"   • Duplicados completos: {df_analisis.duplicated().sum()}")
if valores_nulos.sum() > 0:
    print(f"   • Campos con valores nulos: {(valores_nulos > 0).sum()}")
else:
    print(f"   • Sin valores nulos detectados")

print("\n✅ Dataset listo para análisis más profundos!")
print("="*60)


                    RESUMEN EJECUTIVO
📊 VOLUMEN DE DATOS:
   • Total de opiniones: 1,314
   • Ciudades analizadas: 2
   • Atracciones totales: 20

🏙️ DISTRIBUCIÓN POR CIUDADES:
   • CANCUN: 700 opiniones (53.3%)
   • CDMX: 614 opiniones (46.7%)

⭐ CALIFICACIONES:
   • Promedio general: 4.26/5
   • Mediana: 5.0/5

👥 TIPO DE VIAJE:
   • Más común: Familia

🔍 CALIDAD DE DATOS:
   • Duplicados completos: 0
   • Sin valores nulos detectados

✅ Dataset listo para análisis más profundos!
   • Duplicados completos: 0
   • Sin valores nulos detectados

✅ Dataset listo para análisis más profundos!


In [587]:
# Verificar valores únicos de todas las columnas para el texto consolidado
print("=== VERIFICACIÓN DE VALORES ÚNICOS PARA TEXTO CONSOLIDADO ===")

print("📋 TIPOS DE VIAJE únicos:")
tipos_viaje_unicos = df_opiniones['TipoViaje'].value_counts()
print(tipos_viaje_unicos)

print(f"\n📋 ORIGEN AUTOR (muestra de valores únicos):")
origen_unicos = df_opiniones['OrigenAutor'].value_counts().head(10)
print(origen_unicos)

print(f"\n📋 TÍTULOS (muestra de valores únicos):")
titulos_unicos = df_opiniones['Titulo'].value_counts().head(5)
for titulo, count in titulos_unicos.items():
    print(f"'{titulo[:50]}...': {count} veces")

print(f"\n💡 ANÁLISIS PARA CONSTRUCCIÓN DE TEXTO:")
print("Basándome en estos valores, la lógica será:")
print("• TipoViaje 'Familia' → 'Mi viaje fue con mi familia'")
print("• TipoViaje 'Pareja' → 'Mi viaje fue con mi pareja'") 
print("• TipoViaje 'Amigos' → 'Mi viaje fue con amigos'")
print("• TipoViaje 'Solitario' → 'Mi viaje fue en solitario'")
print("• TipoViaje 'Negocios' → 'Mi viaje fue por negocios'")
print("• TipoViaje 'desconocido' → no agregar nada")
print("• OrigenAutor 'anonimo' → no agregar nada")
print("• Titulo 'sin titulo' → no agregar nada")

=== VERIFICACIÓN DE VALORES ÚNICOS PARA TEXTO CONSOLIDADO ===
📋 TIPOS DE VIAJE únicos:
TipoViaje
Familia        471
Pareja         338
desconocido    203
Amigos         175
Solitario      115
Negocios        12
Name: count, dtype: int64

📋 ORIGEN AUTOR (muestra de valores únicos):
OrigenAutor
anonimo                             518
Cancún, México                       41
Ciudad de México, México             32
Buenos Aires, Argentina              27
Bogotá, Colombia                     21
Montevideo, Uruguay                  19
Santiago, Chile                      11
Estados Unidos                       10
Monterrey, México                    10
Nueva York, Estado de Nueva York      9
Name: count, dtype: int64

📋 TÍTULOS (muestra de valores únicos):
'Excelente...': 14 veces
'Increíble...': 9 veces
'sin titulo...': 8 veces
'Hermoso...': 6 veces
'Maravilloso...': 5 veces

💡 ANÁLISIS PARA CONSTRUCCIÓN DE TEXTO:
Basándome en estos valores, la lógica será:
• TipoViaje 'Familia' → 'Mi viaje 

In [555]:
# Crear columna de texto consolidado
print("=== CREACIÓN DE COLUMNA DE TEXTO CONSOLIDADO ===")
print("Combinando todos los campos en una narrativa coherente...")

def crear_texto_consolidado(row):
    """
    Crea un texto consolidado que combina todos los campos en una narrativa natural.
    
    Formato: [Titulo]. [Review]. Mi viaje fue [en/con] [TipoViaje]. Yo soy de [OrigenAutor].
    """
    
    # Obtener los valores y limpiarlos
    titulo = str(row['Titulo']).strip()
    review = str(row['Review']).strip()
    tipo_viaje = str(row['TipoViaje']).strip()
    origen = str(row['OrigenAutor']).strip()
    
    # Construir el texto consolidado
    texto_partes = []
    
    # 1. Agregar título (si no es "sin titulo")
    if titulo and titulo.lower() != 'sin titulo':
        # Asegurar que termine con punto
        if not titulo.endswith('.'):
            titulo += '.'
        texto_partes.append(titulo)
    
    # 2. Agregar review
    if review and review.lower() not in ['nan', 'none']:
        # Asegurar que termine con punto
        if not review.endswith('.'):
            review += '.'
        texto_partes.append(review)
    
    # 3. Agregar información del tipo de viaje (si no es "desconocido")
    if tipo_viaje and tipo_viaje.lower() != 'desconocido':
        # Mapear cada tipo de viaje específico a su frase correspondiente
        if tipo_viaje.lower() == 'familia':
            texto_partes.append("Mi viaje fue con mi familia.")
        elif tipo_viaje.lower() == 'pareja':
            texto_partes.append("Mi viaje fue con mi pareja.")
        elif tipo_viaje.lower() == 'amigos':
            texto_partes.append("Mi viaje fue con amigos.")
        elif tipo_viaje.lower() == 'solitario':
            texto_partes.append("Mi viaje fue en solitario.")
        elif tipo_viaje.lower() == 'negocios':
            texto_partes.append("Mi viaje fue por negocios.")
        else:
            # Para cualquier otro caso no previsto, usar genérico
            texto_partes.append(f"Mi viaje fue en {tipo_viaje.lower()}.")
    
    # Unir todas las partes con espacios
    texto_consolidado = ' '.join(texto_partes)
    
    return texto_consolidado

# Aplicar la función a todo el dataset
print("Generando texto consolidado para cada registro...")
df_opiniones['texto_consolidado'] = df_opiniones.apply(crear_texto_consolidado, axis=1)

print(f"✅ Columna 'texto_consolidado' creada exitosamente")
print(f"Total de registros procesados: {len(df_opiniones)}")

# Mostrar estadísticas de la nueva columna
longitudes = df_opiniones['texto_consolidado'].str.len()
print(f"\n📊 ESTADÍSTICAS DE TEXTO CONSOLIDADO:")
print(f"   • Longitud promedio: {longitudes.mean():.1f} caracteres")
print(f"   • Longitud mínima: {longitudes.min()} caracteres")
print(f"   • Longitud máxima: {longitudes.max()} caracteres")
print(f"   • Mediana: {longitudes.median():.1f} caracteres")

# Mostrar algunos ejemplos
print(f"\n📝 EJEMPLOS DE TEXTO CONSOLIDADO:")
print("="*80)
ejemplos = df_opiniones.sample(5, random_state=42)
for idx, row in ejemplos.iterrows():
    print(f"\nEjemplo {idx}:")
    print(f"Ciudad: {row['ciudad']} | Atracción: {row['atraccion']}")
    print(f"Texto consolidado: {row['texto_consolidado']}")
    print("-" * 80)

=== CREACIÓN DE COLUMNA DE TEXTO CONSOLIDADO ===
Combinando todos los campos en una narrativa coherente...
Generando texto consolidado para cada registro...
✅ Columna 'texto_consolidado' creada exitosamente
Total de registros procesados: 1314

📊 ESTADÍSTICAS DE TEXTO CONSOLIDADO:
   • Longitud promedio: 366.3 caracteres
   • Longitud mínima: 108 caracteres
   • Longitud máxima: 2259 caracteres
   • Mediana: 259.5 caracteres

📝 EJEMPLOS DE TEXTO CONSOLIDADO:

Ejemplo 1233:
Ciudad: cdmx | Atracción: palacio de bellas artes
Texto consolidado: Murales. Muchos murales y obras de arte de varios artistas mexicanos, incluido Diego Rivera, se muestran en 3 plantas. También exposiciones temporales de otros artistas de otras formas de arte (vidrio, etc). El famoso mural de Diego Rivera "El hombre en la encrucijada" es bastante fresco de ver. El edificio en sí es muy magnífico de ver en sí mismo. Museo es de visita gratuita los domingos, de lo contrario es de $ 95 Pesos Mexicanos. Mi viaje fue con

In [556]:
# Verificar ejemplos específicos del texto consolidado por tipo de viaje
print("=== VERIFICACIÓN DE EJEMPLOS POR TIPO DE VIAJE ===")

tipos_viaje_ejemplos = ['Familia', 'Pareja', 'Amigos', 'Solitario', 'Negocios', 'desconocido']

for tipo in tipos_viaje_ejemplos:
    print(f"\n🔸 EJEMPLOS DE TIPO: {tipo}")
    print("-" * 60)
    
    # Filtrar registros por tipo de viaje
    registros_tipo = df_opiniones[df_opiniones['TipoViaje'] == tipo].head(2)
    
    if len(registros_tipo) > 0:
        for idx, row in registros_tipo.iterrows():
            print(f"Ejemplo {idx}:")
            print(f"  Ciudad/Atracción: {row['ciudad']} - {row['atraccion']}")
            print(f"  Texto consolidado: {row['texto_consolidado'][:200]}...")
            print()
    else:
        print(f"  No se encontraron registros para tipo '{tipo}'")

# Verificar algunos casos específicos de origen
print(f"\n=== VERIFICACIÓN DE EJEMPLOS POR ORIGEN ===")
origenes_especificos = ['anonimo', 'Cancún, México', 'Buenos Aires, Argentina']

for origen in origenes_especificos:
    print(f"\n🔸 EJEMPLOS DE ORIGEN: {origen}")
    print("-" * 60)
    
    registros_origen = df_opiniones[df_opiniones['OrigenAutor'] == origen].head(1)
    
    if len(registros_origen) > 0:
        for idx, row in registros_origen.iterrows():
            print(f"  Texto consolidado: {row['texto_consolidado'][:200]}...")
    else:
        print(f"  No se encontraron registros para origen '{origen}'")

print(f"\n✅ COLUMNA 'texto_consolidado' VERIFICADA Y LISTA")
print(f"📊 Total de registros con texto consolidado: {len(df_opiniones)}")
print(f"📏 Longitud promedio: {df_opiniones['texto_consolidado'].str.len().mean():.1f} caracteres")

=== VERIFICACIÓN DE EJEMPLOS POR TIPO DE VIAJE ===

🔸 EJEMPLOS DE TIPO: Familia
------------------------------------------------------------
Ejemplo 9:
  Ciudad/Atracción: cancun - xoximilco cancun by xcaret
  Texto consolidado: Noche mexicana de fiesta y diversión. Ambiente muy de fiesta mexicana, te permite conocer ciertos platos y dulces tipicos de Mexico, ambiente cool con musica en vivo. Mi viaje fue con mi familia....

Ejemplo 11:
  Ciudad/Atracción: cancun - xoximilco cancun by xcaret
  Texto consolidado: Muy divertido. La experiencia fue súper divertida, lo disfrutamos mucho. Había un poco de calor, por la temporada del año, pero eso no impidió que nos divirtieramos y bailaramos. Mi viaje fue con mi f...


🔸 EJEMPLOS DE TIPO: Pareja
------------------------------------------------------------
Ejemplo 0:
  Ciudad/Atracción: cancun - xoximilco cancun by xcaret
  Texto consolidado: Divertido y emocionante. Se organizó el transporte. La recepción en el parque fue genial con saludos

In [557]:
# Guardar el dataset final con la nueva columna de texto consolidado
print("=== GUARDANDO DATASET FINAL ===")

# Verificar que tenemos todas las columnas esperadas
print("Columnas en el dataset final:")
for i, col in enumerate(df_opiniones.columns, 1):
    print(f"{i:2d}. {col}")

print(f"\nDimensiones finales: {df_opiniones.shape}")
print(f"Filas: {df_opiniones.shape[0]:,}")
print(f"Columnas: {df_opiniones.shape[1]}")

# Guardar el dataset con la nueva columna
df_opiniones.to_csv('../data/dataset_opiniones_consolidado.csv', index=False)

print(f"\n✅ Dataset final guardado como 'dataset_opiniones_consolidado.csv'")
print(f"📊 Incluye la nueva columna 'texto_consolidado' con narrativa completa")

# Resumen final de la nueva columna
longitudes_finales = df_opiniones['texto_consolidado'].str.len()
palabras_finales = df_opiniones['texto_consolidado'].str.split().str.len()

print(f"\n📈 ESTADÍSTICAS FINALES DE TEXTO CONSOLIDADO:")
print(f"   • Longitud promedio: {longitudes_finales.mean():.1f} caracteres")
print(f"   • Palabras promedio: {palabras_finales.mean():.1f} palabras")
print(f"   • Registros procesados: {len(df_opiniones):,}")

print(f"\n🎉 ANÁLISIS EXPLORATORIO COMPLETADO CON ÉXITO!")
print("="*60)

=== GUARDANDO DATASET FINAL ===
Columnas en el dataset final:
 1. Titulo
 2. Review
 3. TipoViaje
 4. Calificacion
 5. OrigenAutor
 6. FechaOpinion
 7. FechaEstadia
 8. ciudad
 9. atraccion
10. texto_consolidado

Dimensiones finales: (1314, 10)
Filas: 1,314
Columnas: 10

✅ Dataset final guardado como 'dataset_opiniones_consolidado.csv'
📊 Incluye la nueva columna 'texto_consolidado' con narrativa completa

📈 ESTADÍSTICAS FINALES DE TEXTO CONSOLIDADO:
   • Longitud promedio: 366.3 caracteres
   • Palabras promedio: 63.0 palabras
   • Registros procesados: 1,314

🎉 ANÁLISIS EXPLORATORIO COMPLETADO CON ÉXITO!

✅ Dataset final guardado como 'dataset_opiniones_consolidado.csv'
📊 Incluye la nueva columna 'texto_consolidado' con narrativa completa

📈 ESTADÍSTICAS FINALES DE TEXTO CONSOLIDADO:
   • Longitud promedio: 366.3 caracteres
   • Palabras promedio: 63.0 palabras
   • Registros procesados: 1,314

🎉 ANÁLISIS EXPLORATORIO COMPLETADO CON ÉXITO!


## 7. Análisis Final Completo - Dataset Limpio

Ahora que hemos completado toda la limpieza y transformación de datos, realizaremos un análisis final completo del dataset limpio para confirmar la calidad de los datos y obtener insights finales.

In [558]:
# Análisis final completo del dataset limpio
print("=" * 80)
print("                     ANÁLISIS FINAL COMPLETO")
print("                    DATASET COMPLETAMENTE LIMPIO")
print("=" * 80)

# 1. Resumen general
print(f"\n📊 RESUMEN GENERAL:")
print(f"   • Total de registros finales: {len(df_opiniones):,}")
print(f"   • Total de columnas: {len(df_opiniones.columns)}")
print(f"   • Ciudades: {df_opiniones['ciudad'].nunique()}")
print(f"   • Atracciones: {df_opiniones['atraccion'].nunique()}")

# 2. Calidad de datos final
print(f"\n🔍 CALIDAD DE DATOS FINAL:")
valores_nulos_final = df_opiniones.isnull().sum().sum()
print(f"   • Valores nulos totales: {valores_nulos_final}")
print(f"   • Duplicados restantes: {df_opiniones.duplicated().sum()}")
print(f"   • Integridad de datos: {'✅ PERFECTA' if valores_nulos_final == 0 else '⚠️ REVISAR'}")

# 3. Distribución por ciudades (final)
print(f"\n🏙️ DISTRIBUCIÓN FINAL POR CIUDADES:")
dist_ciudades_final = df_opiniones['ciudad'].value_counts()
for ciudad, cantidad in dist_ciudades_final.items():
    porcentaje = (cantidad / len(df_opiniones)) * 100
    print(f"   • {ciudad.upper()}: {cantidad:,} ({porcentaje:.1f}%)")

# 4. Análisis de calificaciones (final)
print(f"\n⭐ ANÁLISIS DE CALIFICACIONES FINAL:")
print(f"   • Promedio general: {df_opiniones['Calificacion'].mean():.2f}/5")
print(f"   • Mediana: {df_opiniones['Calificacion'].median():.1f}/5")
print(f"   • Moda: {df_opiniones['Calificacion'].mode()[0]}/5")
print(f"   • Desviación estándar: {df_opiniones['Calificacion'].std():.2f}")

# 5. Distribución de calificaciones
print(f"\n   Distribución de calificaciones:")
dist_calificaciones = df_opiniones['Calificacion'].value_counts().sort_index()
for cal, count in dist_calificaciones.items():
    porcentaje = (count / len(df_opiniones)) * 100
    print(f"     {cal} estrellas: {count:,} ({porcentaje:.1f}%)")

# 6. Análisis temporal final
print(f"\n📅 ANÁLISIS TEMPORAL FINAL:")
rango_fechas = df_opiniones['FechaOpinion'].dropna()
if len(rango_fechas) > 0:
    fecha_min = rango_fechas.min()
    fecha_max = rango_fechas.max()
    print(f"   • Rango de fechas: {fecha_min.strftime('%d/%m/%Y')} - {fecha_max.strftime('%d/%m/%Y')}")
    print(f"   • Período cubierto: {(fecha_max - fecha_min).days} días")
    
    # Análisis por año
    print(f"   • Distribución por año:")
    dist_años = df_opiniones['FechaOpinion'].dt.year.value_counts().sort_index()
    for año, count in dist_años.items():
        porcentaje = (count / len(df_opiniones)) * 100
        print(f"     {año}: {count:,} opiniones ({porcentaje:.1f}%)")

# 7. Tipos de viaje final
print(f"\n👥 TIPOS DE VIAJE FINAL:")
dist_tipos_final = df_opiniones['TipoViaje'].value_counts()
for tipo, count in dist_tipos_final.items():
    porcentaje = (count / len(df_opiniones)) * 100
    print(f"   • {tipo}: {count:,} ({porcentaje:.1f}%)")

# 8. Análisis de texto consolidado
print(f"\n📝 ANÁLISIS DE TEXTO CONSOLIDADO:")
longitud_texto = df_opiniones['texto_consolidado'].str.len()
palabras_texto = df_opiniones['texto_consolidado'].str.split().str.len()
print(f"   • Longitud promedio: {longitud_texto.mean():.1f} caracteres")
print(f"   • Palabras promedio: {palabras_texto.mean():.1f} palabras")
print(f"   • Texto más corto: {longitud_texto.min()} caracteres")
print(f"   • Texto más largo: {longitud_texto.max()} caracteres")

# 9. Top atracciones por calificación
print(f"\n🏆 TOP 5 ATRACCIONES POR CALIFICACIÓN PROMEDIO:")
top_atracciones = df_opiniones.groupby(['ciudad', 'atraccion']).agg({
    'Calificacion': ['mean', 'count']
}).round(2)
top_atracciones.columns = ['calificacion_promedio', 'num_opiniones']
top_atracciones = top_atracciones[top_atracciones['num_opiniones'] >= 5]  # Al menos 5 opiniones
top_atracciones = top_atracciones.sort_values('calificacion_promedio', ascending=False).head()

for (ciudad, atraccion), row in top_atracciones.iterrows():
    print(f"   • {atraccion} ({ciudad}): {row['calificacion_promedio']:.2f}/5 ({int(row['num_opiniones'])} opiniones)")

# 10. Origen de autores más frecuentes (excluyendo anónimos)
print(f"\n🌍 TOP 10 PAÍSES/REGIONES DE ORIGEN:")
origenes_top = df_opiniones[df_opiniones['OrigenAutor'] != 'anonimo']['OrigenAutor'].value_counts().head(10)
for origen, count in origenes_top.items():
    porcentaje = (count / len(df_opiniones)) * 100
    print(f"   • {origen}: {count:,} ({porcentaje:.1f}%)")

print(f"\n" + "=" * 80)
print("✅ ANÁLISIS FINAL COMPLETADO")
print("🎯 Dataset completamente limpio y listo para análisis avanzados")
print("📊 Calidad de datos: EXCELENTE")
print("=" * 80)

                     ANÁLISIS FINAL COMPLETO
                    DATASET COMPLETAMENTE LIMPIO

📊 RESUMEN GENERAL:
   • Total de registros finales: 1,314
   • Total de columnas: 10
   • Ciudades: 2
   • Atracciones: 20

🔍 CALIDAD DE DATOS FINAL:
   • Valores nulos totales: 0
   • Duplicados restantes: 0
   • Integridad de datos: ✅ PERFECTA

🏙️ DISTRIBUCIÓN FINAL POR CIUDADES:
   • CANCUN: 700 (53.3%)
   • CDMX: 614 (46.7%)

⭐ ANÁLISIS DE CALIFICACIONES FINAL:
   • Promedio general: 4.26/5
   • Mediana: 5.0/5
   • Moda: 5/5
   • Desviación estándar: 1.13

   Distribución de calificaciones:
     1 estrellas: 78 (5.9%)
     2 estrellas: 34 (2.6%)
     3 estrellas: 145 (11.0%)
     4 estrellas: 269 (20.5%)
     5 estrellas: 788 (60.0%)

📅 ANÁLISIS TEMPORAL FINAL:
   • Rango de fechas: 11/04/2018 - 01/09/2025
   • Período cubierto: 2700 días
   • Distribución por año:
     2018: 17 opiniones (1.3%)
     2019: 62 opiniones (4.7%)
     2020: 50 opiniones (3.8%)
     2021: 39 opiniones (3.0%)
 

In [559]:
df_opiniones.head()

Titulo  \
0                         Divertido y emocionante   
1                                       Increíble   
2                Xochimilco excelente experiencia   
3                 Xoximilco increible experiencia   
4  Xochimilco tiene el mejor ambiente de fiesta!!   

                                              Review TipoViaje  Calificacion  \
0  Se organizó el transporte. La recepción en el ...    Pareja             5   
1  Ninguno, me encantó toda la experiencias fue i...    Pareja             5   
2  Joss y Roberto dieron muy buen servicio y ambi...    Pareja             5   
3  Fue una experiencia maravillosa el ambiente de...    Amigos             5   
4  Fui con mis amigos a celebrar mi cumpleaños y ...    Amigos             5   

     OrigenAutor FechaOpinion FechaEstadia  ciudad  \
0        anonimo   2025-08-31   2025-08-01  cancun   
1        anonimo   2025-08-28   2025-08-01  cancun   
2        anonimo   2025-08-22   2025-08-01  cancun   
3        anonimo   2025-08-21   2025-08-01  cancun   
4  María Elena F   2025-08-19   2025-08-01  cancun   

                    atraccion  \
0  xoximilco cancun by xcaret   
1  xoximilco cancun by xcaret   
2  xoximilco cancun by xcaret   
3  xoximilco cancun by xcaret   
4  xoximilco cancun by xcaret   

                                   texto_consolidado  
0  Divertido y emocionante. Se organizó el transp...  
1  Increíble. Ninguno, me encantó toda la experie...  
2  Xochimilco excelente experiencia. Joss y Rober...  
3  Xoximilco increible experiencia. Fue una exper...  
4  Xochimilco tiene el mejor ambiente de fiesta!!...